В данном датасете построим некоторую базовую модель, относительно предсказаний которой можно будет отталкиваться в оценке точности.

Устроим блиц-проверку алгоритмов sklearn по мотивам статьи https://habr.com/ru/post/475552/

По условиям соревнования, критерием оценки будет MAPE. Мы тоже будет его использовать.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lars
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LarsCV
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from matplotlib import pyplot

# Загрузка тренировочного датасета

In [3]:
X = pd.read_csv('EDAv1_Train.zip').drop(columns='price')
X.head(1).T

,0
body_type,седан
brand,AUDI
color,чёрный
fuel_type,бензин
model_year,1990
n_doors,4
production_year,1991
vehicle_transmission,механика
engine_power,174.0
mileage,350000


In [4]:
y = pd.read_csv('EDAv1_Train.zip', usecols=['price'])
y.head(1)

,price
0,200000.0


# Проверим готовность данных

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86853 entries, 0 to 86852
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   body_type             86853 non-null  object 
 1   brand                 86853 non-null  object 
 2   color                 86853 non-null  object 
 3   fuel_type             86853 non-null  object 
 4   model_year            86853 non-null  int64  
 5   n_doors               86853 non-null  int64  
 6   production_year       86853 non-null  int64  
 7   vehicle_transmission  86853 non-null  object 
 8   engine_power          86853 non-null  float64
 9   mileage               86853 non-null  int64  
 10  drive_type            86853 non-null  object 
 11  n_owners              86853 non-null  object 
 12  model_name            86853 non-null  object 
 13  is_original_techpass  86853 non-null  int64  
 14  is_lefthand_drive     86853 non-null  int64  
 15  engine_displacement

В обоих датасетах есть пропуски в объёме двигателя. 
При этом, во время EDA было видно, что у объема высокая корреляция с мощностью двигателя.
Поэтому, в данной ситуации, для baseline удалим этот признак.

Также удалим признак model_year - высокая корреляция с production_year, который д.б. более важным по смыслу.

Признак model_name тоже пока что удалим - слишком много вариаций.

In [6]:
columns_to_drop = ['engine_displacement', 'model_year', 'model_name']
X.drop(columns=columns_to_drop, inplace=True)

In [7]:
# one-hot-encoding для категориальных признаков
X = pd.get_dummies(X)

# Проверяем модели

## Разделяем на тренировочную и валидационную части

In [23]:
# параметры разделения
test_size = 0.3
seed = 73

In [24]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=seed)
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

(60797, 80) (60797, 1)
(26056, 80) (26056, 1)


In [25]:
# параметры оценки
num_folds = 5
n_iter = 1000
n_estimators = 100
scoring = 'neg_mean_absolute_percentage_error'

In [26]:
# список моделей-кандидатов
# закомментированы совсем плохие по результатам или расходящиеся
models = []
models.append(('LR', LinearRegression(n_jobs=-1)))
models.append(('R', Ridge()))
# models.append(('L', Lasso()))
# models.append(('ELN', ElasticNet()))
# models.append(('LARS', Lars(normalize=False)))
# models.append(('BR', BayesianRidge(n_iter=n_iter)))
models.append(('KNR', KNeighborsRegressor(n_jobs=-1))) # более-менее результат, но хуже ансамблей и очень медленно
models.append(('DTR', DecisionTreeRegressor()))
# models.append(('LSVR', LinearSVR()))
# models.append(('SVR', SVR()))
# models.append(('ABR', AdaBoostRegressor(n_estimators=n_estimators))) # очень плохие результаты
models.append(('BR', BaggingRegressor(n_estimators=n_estimators, n_jobs=-1)))
models.append(('ETR', ExtraTreesRegressor(n_estimators=n_estimators, n_jobs=-1)))
models.append(('GBR', GradientBoostingRegressor(n_estimators=n_estimators)))
models.append(('RFR', RandomForestRegressor(n_estimators=n_estimators, n_jobs=-1)))

In [27]:
# таблица результатов
models_effcy = pd.DataFrame(index=[name for name, model in models])

In [28]:
# Оценивание эффективности каждой модели
for name, model in models:
    # оценка с отложенной частью
    m_fit = model.fit(X_train, np.ravel(y_train))
    y_predict = model.predict(X_valid)
    m_score = mean_absolute_percentage_error(y_valid, y_predict)
    models_effcy.loc[name, 'split_score'] = m_score
    # оценка кросс-валидацией по всем данным
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)
    cv_results = cross_val_score(model, X, np.ravel(y.values), cv=kfold, scoring=scoring)
    models_effcy.loc[name, 'cv_mean'] = cv_results.mean()
    models_effcy.loc[name, 'cv_std'] = cv_results.std()
    # models_effcy.loc[name, 'results'] = list(cv_results)
    print(name, model, 'complete')
display(models_effcy)

LR LinearRegression(n_jobs=-1) complete
R Ridge() complete
KNR KNeighborsRegressor(n_jobs=-1) complete
DTR DecisionTreeRegressor() complete
BR BaggingRegressor(n_estimators=100, n_jobs=-1) complete
ETR ExtraTreesRegressor(n_jobs=-1) complete
GBR GradientBoostingRegressor() complete
RFR RandomForestRegressor(n_jobs=-1) complete


,split_score,cv_mean,cv_std
LR,0.938004,-0.957173,0.010086
R,0.938014,-0.957154,0.010070
KNR,0.414946,-0.412539,0.003390
DTR,0.201589,-0.201753,0.003571
BR,0.158559,-0.157246,0.002921
ETR,0.159024,-0.159154,0.002644
GBR,0.263766,-0.275265,0.007704
RFR,0.158183,-0.157090,0.003069


split_score - чем меньше, тем лучше

cv_mean - чем меньше по модулю, тем лучше

По показателям выберем BaggingRegressor для baseline

ТОП для тюнинга и стекинга:
- KNeighborsRegressor
- BaggingRegressor
- RandomForestRegressor
- ExtraTreesRegressor
- DecisionTreeRegressor

# Результат для Kaggle

In [39]:
X_train = pd.read_csv('EDAv1_Train.zip').drop(columns='price')
y_train = pd.read_csv('EDAv1_Train.zip', usecols=['price'])
X_test = pd.read_csv('EDAv1_Test.zip')

In [40]:
# склеим датасеты признаков
X_train['is_train'] = 1
X_test['is_train'] = 0
X = X_train.append(X_test)

In [41]:
# обработаем также
columns_to_drop = ['engine_displacement', 'model_year', 'model_name']
X.drop(columns=columns_to_drop, inplace=True)
X = pd.get_dummies(X)

In [42]:
# разделим
X_train = X[X['is_train'] == 1].drop(columns=['is_train'])
X_test = X[X['is_train'] == 0].drop(columns=['is_train'])

In [44]:
# обучим
baseline_model = BaggingRegressor(n_estimators=n_estimators, n_jobs=-1)
baseline_model.fit(X_train, np.ravel(y_train))

BaggingRegressor(n_estimators=100, n_jobs=-1)

In [45]:
# предскажем
y_test_predict = baseline_model.predict(X_test)

In [47]:
submission = pd.read_csv('sample_submission_empty.csv')
submission['price'] = y_test_predict
submission.to_csv('sample_submission.csv', index=False)